In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from math import pi, ceil
plt.style.use('seaborn')
sns.set_style("whitegrid")

import urllib.request
from nltk.tokenize import RegexpTokenizer

import re
import random
import numpy as np
import pandas as pd
import random
import math
import itertools
import multiprocessing
from tqdm import tqdm
import logging
import pickle

from scipy.sparse.linalg import svds,eigs
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
from scipy import stats

from time import time
import warnings
warnings.filterwarnings(action='ignore')

import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GIUCO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
track = pd.read_csv('tracks.csv')
track = pd.DataFrame(track)
print(track.shape)
track.head()

(1472, 22)


,Unnamed: 0,id,artist_name,track_name,album_name,artist_genre,release_date,artist_popularity,track_popularity,artist_followers,...,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,3nqQXoyQOWXiESFLlDF1hG,Sam Smith,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),"['dance pop', 'pop', 'uk pop']",2022-09-22,88,100,20816519,...,2,-7.375,0.0864,0.01300,0.000005,0.266,0.238,131.121,156943,4
1,1,0V3wPSX9ygBnCm8psDIegu,Taylor Swift,Anti-Hero,Midnights,['pop'],2022-10-21,100,97,62687649,...,4,-6.571,0.0519,0.13000,0.000002,0.142,0.533,97.008,200690,4
2,2,1bDbXMyjaUIooNwFE9wn0N,Drake,Rich Flex,Her Loss,"['canadian hip hop', 'canadian pop', 'hip hop'...",2022-11-04,97,95,69088297,...,11,-9.342,0.2440,0.05030,0.000002,0.355,0.424,153.150,239360,3
3,3,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,La Bachata,"['colombian pop', 'latin pop', 'reggaeton', 'r...",2022-05-26,84,97,10812997,...,7,-5.329,0.0364,0.58300,0.000002,0.218,0.850,124.980,162638,4
4,4,4uUG5RXrOk84mYEfFvj3cK,David Guetta,I'm Good (Blue),I'm Good (Blue),"['big room', 'dance pop', 'edm', 'pop', 'pop d...",2022-08-26,88,97,25038984,...,7,-3.673,0.0343,0.00383,0.000007,0.371,0.304,128.040,175238,4


In [13]:
def cleanText(readData):
    text = re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData)
 
    return text

In [14]:
genre = []
for i in track['artist_genre']:
    if i == '[]':
        i = 'NA'
        genre.append(i.strip())
    else:
        i = cleanText(i)
        genre.append(i.strip())
track['genre'] = genre


In [15]:
track = track[track['genre'] != "NA"]
track = track.reset_index()
track['track_popularity'] = track['track_popularity'] / 100 

In [16]:
from ContentTFIDF import ContentTFIDF
ct = ContentTFIDF(track)
ct_tfidf = ct.calculateTFIDF()
ct_tfidf.shape


(1282, 1446)

In [17]:
from ContentBasedRecommender import ContentBasedRecommender

a = ContentBasedRecommender(track, ct_tfidf)
b = a.user_info()
c = a.recommend_features()
d = a.recommend_genre()
e = a.feature_genre_intersection(track, c,d)
f = a.get_total_score()
f

What's your favourite song out of these?   1) Chandelier  2) Why'd You Only Call Me When You're High?  3) Erase Your Social  4) 初めの一歩  5) Out There
Light or dark song?   1) light  2) dark
Fast songs or slow songs?   1) fast  2) slow
What's your current mood?   1) happy  2) meh  3) sad


,artist_name,track_name,album_name,mood_score,speed_score,emotion_score,genre_score,total_score
30,Arctic Monkeys,R U Mine?,AM,0.381,0.670566,0.262670,1.000000,0.719039
11,Wallows,Remember When,Nothing Happens,0.490,0.911652,0.431040,0.085859,0.348378
15,Marcy Playground,Sex & Candy,Marcy Playground,0.370,0.935593,0.499587,0.000000,0.300863
17,Fugazi,Waiting Room,13 Songs,0.460,0.747838,0.366002,0.000000,0.262307
5,Red Hot Chili Peppers,Scar Tissue,Californication (Deluxe Edition),0.453,0.800419,0.253553,0.000000,0.251162
9,K.Flay,High Enough,Every Where Is Some Where,0.457,0.800179,0.218277,0.000000,0.245909
26,Vista Kicks,Marceline,Chasing Waves,0.415,0.718783,0.279555,0.000000,0.235556
28,The Technicolors,Tonight You Are Mine,Ultraviolet Disguise (Deluxe Edition),0.360,0.783670,0.220855,0.000000,0.227421
14,Joy Division,Disorder - 2007 Remaster,Unknown Pleasures,0.260,0.868709,0.223309,0.000000,0.225336
31,Miniature Tigers,Like or Like Like,Tell It to the Volcano,0.162,0.686820,0.488955,0.000000,0.222962
